In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score  
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import torch


In [ ]:
x_trainval_mapping_df = pd.read_csv('..data/x_trainval_mapping_tail.csv').drop('Unnamed: 0',axis=1)
x_test_mapping_df=pd.read_csv('..data/x_test_map_tail.csv').drop('Unnamed: 0',axis=1)
x_trainval_mapping_df

,color,fixed_acidity2,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,color:White,fixed_acidity2:6.9,volatile_acidity:0.38,citric_acid:0.25,residual_sugar:9.8,chlorides:0.04,free_sulfur_dioxide:28.0,total_sulfur_dioxide:191.0,density:0.9971,pH:3.28,sulphates:0.61,alcohol:9.2
1,color:White,fixed_acidity2:6.0,volatile_acidity:0.19,citric_acid:0.29,residual_sugar:1.1,chlorides:0.047,free_sulfur_dioxide:67.0,total_sulfur_dioxide:152.0,density:0.9916,pH:3.54,sulphates:0.59,alcohol:11.1
2,color:White,fixed_acidity2:6.1,volatile_acidity:0.24,citric_acid:0.32,residual_sugar:9.0,chlorides:0.031,free_sulfur_dioxide:41.0,total_sulfur_dioxide:134.0,density:0.99234,pH:3.25,sulphates:0.26,alcohol:12.3
3,color:White,fixed_acidity2:7.7,volatile_acidity:0.16,citric_acid:0.41,residual_sugar:1.7,chlorides:0.048,free_sulfur_dioxide:60.0,total_sulfur_dioxide:173.0,density:0.9932,pH:3.24,sulphates:0.66,alcohol:11.2
4,color:White,fixed_acidity2:7.6,volatile_acidity:0.26,citric_acid:0.47,residual_sugar:1.6,chlorides:0.068,free_sulfur_dioxide:5.0,total_sulfur_dioxide:55.0,density:0.9944,pH:3.1,sulphates:0.45,alcohol:9.6
...,...,...,...,...,...,...,...,...,...,...,...,...
3110,color:Red,fixed_acidity2:7.5,volatile_acidity:0.77,citric_acid:0.2,residual_sugar:8.1,chlorides:0.098,free_sulfur_dioxide:30.0,total_sulfur_dioxide:92.0,density:0.99892,pH:3.2,sulphates:0.58,alcohol:9.2
3111,color:White,fixed_acidity2:7.5,volatile_acidity:0.23,citric_acid:0.3,residual_sugar:1.2,chlorides:0.03,free_sulfur_dioxide:27.0,total_sulfur_dioxide:80.0,density:0.99192,pH:3.05,sulphates:0.68,alcohol:10.5
3112,color:White,fixed_acidity2:6.6,volatile_acidity:0.545,citric_acid:0.04,residual_sugar:2.5,chlorides:0.031,free_sulfur_dioxide:48.0,total_sulfur_dioxide:111.0,density:0.9906,pH:3.14,sulphates:0.32,alcohol:11.9
3113,color:White,fixed_acidity2:7.2,volatile_acidity:0.13,citric_acid:0.46,residual_sugar:1.3,chlorides:0.044000000000000004,free_sulfur_dioxide:48.0,total_sulfur_dioxide:111.0,density:0.9912700000000001,pH:2.97,sulphates:0.45,alcohol:11.1


In [ ]:
rela_embedding=pd.read_csv('..data/KRAL_Embedding_Model_WINE_D032-rela.csv')
tail_embedding=pd.read_csv('..data/KRAL_Embedding_Model_WINE_D032-tail.csv')
x_test = pd.read_csv("..data/wine_x_test.csv").drop('Unnamed: 0',axis=1)
wine_train_val=pd.read_csv("..data/wine_train_val.csv").drop('Unnamed: 0',axis=1)
y_train_val=pd.read_csv("..data/y_wine_trainval.csv").drop('Unnamed: 0',axis=1)
y_test=pd.read_csv("..data/y_test_wine.csv").drop('Unnamed: 0',axis=1)


In [ ]:
test_notnull=[]
for i in range(len(x_test_mapping_df)):
    total_test_reltal=[]
    for j in x_test_mapping_df.columns:
        #rela=rela_embedding[rela_embedding.relation==j].drop(['relation'],axis=1).values.tolist()[0]
        tailv=tail_embedding[tail_embedding['tail']==x_test_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_test_reltal.append((tailv[0]))
        else:
          total_test_reltal.append([0]*32)

    test_notnull.append(total_test_reltal)
      

test_notnull_len=[]
for i in range(len(test_notnull)):
    test_notnull_len.append(len(test_notnull[i]))

In [ ]:
train_val_notnulllst=[]
for i in range(len(x_trainval_mapping_df)):
    total_trainval_reltal=[]
    for j in x_trainval_mapping_df.columns:
        #rela=rela_embedding[rela_embedding.relation==j].drop(['relation'],axis=1).values.tolist()[0]
        tailv=tail_embedding[tail_embedding['tail']==x_trainval_mapping_df.loc[i,j]].drop(['tail'],axis=1).values.tolist()
        if tailv:
          total_trainval_reltal.append((tailv[0]))
        else:
          total_trainval_reltal.append([0]*32)
    train_val_notnulllst.append(total_trainval_reltal)
      
trainval_notnull_len=[]
for i in range(len(train_val_notnulllst)):
    trainval_notnull_len.append(len(train_val_notnulllst[i]))

In [ ]:
print(len(train_val_notnulllst))

print(len(train_val_notnulllst[1]))
print(len(train_val_notnulllst[1][0]))


3115
12
32


In [ ]:
import pickle
filename = './wine_test_onlytail.pickle'
trainval_file = './wine_trainval_onlytail.pickle'
# open the file for writing in binary mode
with open(filename, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(test_notnull, f)
with open(trainval_file, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(train_val_notnulllst, f)

In [ ]:
import random
from sklearn.model_selection import train_test_split
train_val_data=train_val_notnulllst 
train_val_label=y_train_val.values.tolist()

train_data, val_data, train_y, val_y =  train_test_split(train_val_data,train_val_label,test_size=0.3, stratify=train_val_label, random_state=0)

print(len(train_val_label))
print(len(train_data))
print(len(val_data))


3115
2180
935


In [ ]:
import pickle
train_data_f = './tailemb_train_data.pickle'
val_data_f = './tailemb_val_data.pickle'
# open the file for writing in binary mode
with open(train_data_f, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(train_data, f)
with open(val_data_f, 'wb') as f:
    # serialize the list and write it to the file
    pickle.dump(val_data, f)




In [ ]:
pd.DataFrame(sum(train_y, [])).to_csv('./train_y.csv',index=0)
pd.DataFrame(sum(val_y, [])).to_csv('./val_y.csv',index=0)


In [ ]:
import pickle

train_data_f = './tailemb_train_data.pickle'
val_data_f = './tailemb_val_data.pickle'
test_f = './wine_test_onlytail.pickle'

with open(train_data_f, 'rb') as f:
    # deserialize the list and load it from the file
    train_data = pickle.load(f)
with open(val_data_f, 'rb') as f:
    # deserialize the list and load it from the file
    val_data = pickle.load(f)

with open(test_f, 'rb') as f:
    # deserialize the list and load it from the file
    test_notnull = pickle.load(f)

train_y = pd.read_csv('./train_y.csv')
val_y=pd.read_csv('./val_y.csv')
train_y


,0
0,0
1,0
2,0
3,1
4,0
...,...
2175,0
2176,1
2177,0
2178,0


**in order to input train_data and val_data into xgboost(2-D data), flatten train_data and val_data**

In [ ]:
train_flatten = np.reshape(train_data, (len(train_data), 12*32))
val_flatten=np.reshape(val_data, (len(val_data), 12*32))
test_flatten = np.reshape(test_notnull, (len(test_notnull), 12*32))
print(train_flatten.shape)
print(val_flatten.shape)
print(test_flatten.shape)


(2180, 384)
(935, 384)
(546, 384)


In [ ]:
model = XGBClassifier(
    max_depth=15,
    learning_rate=0.1,
    n_estimators=300,
    gamma=0.1,
    subsample=0.8,
    colsample_bytree=0.7,
    scale_pos_weight=1
)

model.fit(train_flatten, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:

y_predict=model.predict_proba(val_flatten)[:, 1]

#find the optimal threshold
precision, recall, thresholds = precision_recall_curve(val_y, y_predict)

df_recall_precision = pd.DataFrame({'Precision':precision[:-1],
                                    'Recall':recall[:-1],
                                    'Threshold':thresholds})


np.seterr(divide='ignore', invalid='ignore')
f1_score = (2 * precision * recall) / (precision + recall)
findex=list(f1_score).index(max(f1_score))
thresholdOpt = round(thresholds[findex], ndigits = 4)
fscoreOpt = round(f1_score[findex], ndigits = 4)
recallOpt = round(recall[findex], ndigits = 4)
precisionOpt = round(precision[findex], ndigits = 4)
print('Best Threshold: {} , F-Score: {}'.format(thresholdOpt, fscoreOpt))
print('Recall: {}, Precision: {}'.format(recallOpt, precisionOpt))

from sklearn.metrics import precision_score, classification_report, recall_score, f1_score, roc_auc_score, confusion_matrix, accuracy_score
y_pre_test=model.predict_proba(test_flatten)[:, 1]
defaulter_decision = (y_pre_test >= thresholdOpt)
print(classification_report(y_test, defaulter_decision))






Best Threshold: 0.5604000091552734 , F-Score: 0.8421
Recall: 0.8563, Precision: 0.8284
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       358
           1       0.84      0.80      0.82       188

    accuracy                           0.88       546
   macro avg       0.87      0.86      0.87       546
weighted avg       0.88      0.88      0.88       546

